<a href="https://colab.research.google.com/github/yarusx/cat_vs_dogo/blob/main/dogs_breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -q /content/drive/MyDrive/Cat.zip

In [4]:
data_dir = pathlib.Path('/content/Cat')

In [8]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

In [9]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='int',
  class_names = ['egyptian_mau', 'siamese', 'birman', 'abyssinian', 'sphynx', 'british_shorthair', 'bombay', 'russian_blue', 'persian', 'ragdoll', 'maine_coon', 'bengal'],
  validation_split=0.2,
  subset="training",
  seed=51,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='int',
  class_names = ['egyptian_mau', 'siamese', 'birman', 'abyssinian', 'sphynx', 'british_shorthair', 'bombay', 'russian_blue', 'persian', 'ragdoll', 'maine_coon', 'bengal'],
  validation_split=0.2,
  subset="validation",
  seed=51,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

Found 2400 files belonging to 12 classes.
Using 1920 files for training.
Found 2400 files belonging to 12 classes.
Using 480 files for validation.


In [10]:
class_names = train_dataset.class_names

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
# val_batches = tf.data.experimental.cardinality(validation_dataset)
# test_dataset = validation_dataset.take(val_batches // 5)
# validation_dataset = validation_dataset.skip(val_batches // 5)

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [12]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
# for image, _ in train_dataset.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

In [13]:
#preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
preprocess_input = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)

In [14]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [15]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [16]:
base_model.trainable = False

In [17]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_avg = global_avg_layer(feature_batch)
print(feature_batch_avg.shape)

(32, 1280)


In [18]:
prediction_layer = tf.keras.layers.Dense(37)
prediction_batch = prediction_layer(feature_batch_avg)
print(prediction_batch.shape)

(32, 37)


In [19]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_avg_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
loss0, accuracy0 = model.evaluate(validation_dataset)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

15/15 [==============================] - 1s 47ms/step - loss: 4.1803 - accuracy: 0.0375
initial loss: 4.18
initial accuracy: 0.04


In [23]:
initial_epochs = 1
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)
val_acc = history.history['val_accuracy']

60/60 [==============================] - 6s 102ms/step - loss: 3.4354 - accuracy: 0.1016 - val_loss: 2.6316 - val_accuracy: 0.2146


In [ ]:
while np.mean(val_acc)*100 < 90.5:
  initial_epochs = 5
  history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)
  
  val_acc = history.history['val_accuracy']
  
model.save("cats_breeds.pb")

from google.colab import files
files.download("cats_breeds.pb")

Epoch 1/5
60/60 [==============================] - 6s 104ms/step - loss: 2.4677 - accuracy: 0.2229 - val_loss: 2.1440 - val_accuracy: 0.3146
Epoch 2/5
60/60 [==============================] - 6s 102ms/step - loss: 2.0535 - accuracy: 0.3349 - val_loss: 1.8169 - val_accuracy: 0.4396
Epoch 3/5
60/60 [==============================] - 6s 102ms/step - loss: 1.7279 - accuracy: 0.4281 - val_loss: 1.5754 - val_accuracy: 0.5229
Epoch 4/5
60/60 [==============================] - 6s 101ms/step - loss: 1.5052 - accuracy: 0.5151 - val_loss: 1.3981 - val_accuracy: 0.5833
Epoch 5/5
60/60 [==============================] - 6s 100ms/step - loss: 1.3421 - accuracy: 0.5635 - val_loss: 1.2599 - val_accuracy: 0.6396
Epoch 1/5
60/60 [==============================] - 6s 103ms/step - loss: 1.2039 - accuracy: 0.6245 - val_loss: 1.1534 - val_accuracy: 0.6646
Epoch 2/5
60/60 [==============================] - 6s 102ms/step - loss: 1.1076 - accuracy: 0.6552 - val_loss: 1.0619 - val_accuracy: 0.6854
Epoch 3/5
60/

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/cats_breeds.pb')

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(160, 160))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 160, 160, 3)
	# center pixel data
	#img = preprocess_input(img)
	return img

img = load_image('/content/drive/MyDrive/cat_1.jpeg')

In [ ]:
labels = ['egyptian_mau', 'siamese', 'birman', 'abyssinian', 'sphynx', 'british_shorthair', 'bombay', 'russian_blue', 'persian', 'ragdoll', 'maine_coon', 'bengal']

In [ ]:
prediction = model.predict(img, batch_size=None,)
prediction = tf.nn.softmax(prediction)
prediction = np.argmax(prediction, axis=1)
print(prediction)

In [ ]:
plt.figure()
plt.imshow(img[0]/255)
plt.title(labels[int(prediction)])

In [ ]:
# #Retrieve a batch of images from the test set
# image_batch, label_batch = test_dataset.as_numpy_iterator().next()
# predictions = model.predict_on_batch(image_batch)

# # Apply a sigmoid since our model returns logits
# predictions = tf.nn.softmax(predictions)
# #print(predictions.shape)
# predictions = np.argmax(predictions, axis=1)

# print('Predictions:\n', predictions)
# print('Labels:\n', label_batch)

# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   plt.title(class_names[predictions[i]])
#   plt.axis("off")